In [25]:
import csv
import requests
from datetime import datetime
import re

def ler_dados_csv():
    caminho_arquivo = './data/patients.csv'

    dados_pacientes = []

    with open(caminho_arquivo, newline='', encoding='UTF-8') as arquivo_csv:
        leitor_csv = csv.DictReader(arquivo_csv)
        for linha in leitor_csv:
            dados_pacientes.append(linha)

    return dados_pacientes

In [26]:
dados = ler_dados_csv()

In [98]:
def operador(observacao):
    saida = observacao.split('|')
    return saida

def get_id (CPF):
    url_get_id_pacient = 'http://0.0.0.0:8080/fhir/Patient?identifier='+CPF
    retorno = requests.get(url_get_id_pacient)
    retorno_json = retorno.json()
    _id = retorno_json.get('entry')[0].get('resource').get('id')
    return _id


def post_observation(id, observacao):
    url_observation = 'http://localhost:8080/fhir/Observation'
    body = {
        "resourceType": "Observation",
        "type": {
            "coding": [
                {
                    "system": "http://0.0.0.0/CodeSystem/",
                    "display": str(observacao)
                }
            ]
        },
        "subject": {
            "reference": "Patient/"+id
        }
    }
    response = requests.post(url_observation, json=body)

    if response.status_code == 201:
        print(f"Adicionado com sucesso ")
    else:
        print(f"""Falha ao criar""")


def post_condition(id, condition):
    url_observation = 'http://localhost:8080/fhir/Condition'
    body = {
        "resourceType": "Condition",
        "type": {
            "coding": [
                {
                    "system": "http://0.0.0.0/CodeSystem/",
                    "display": str(condition)
                }
            ]
        },
        "subject": {
            "reference": "Patient/"+id
        }
    }
    response = requests.post(url_observation, json=body)

    if response.status_code == 201:
        print(f"Adicionado com sucesso ")
    else:
        print(f"""Falha ao criar""")


for paciente in dados:
    if(paciente['Observacao']):
        observacao = paciente['Observacao'].split('|')
        _id = get_id (paciente['CPF'])
        for condicao in observacao:
            if (condicao == 'Gestante'):
                post_observation(_id, condicao)
            else:
                post_condition(_id,condicao)




Gestante
Adicionado com sucesso 
Gestante
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Gestante
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Adicionado com sucesso 
Gestante
Adicionado com sucesso 


In [38]:
def parse_gender(gender):
    if gender == 'Masculino':
        return 'male'
    if gender == 'Feminino':
        return 'female'
def remover(nome_completo):
    return re.sub(r'\b(?:da|de)\s(?=[A-Z][a-z])', '', nome_completo)

def obter_nomes(nome_completo):
    nome_completo = remover(nome_completo)
    partes_nome = nome_completo.split()

    if len(partes_nome) >= 3:
        primeiros_nomes = ' '.join(partes_nome[:2])
        sobrenomes = ' '.join(partes_nome[2:])
        return primeiros_nomes, sobrenomes
    elif len(partes_nome) == 2:
        primeiros_nomes = ' '.join(partes_nome[:1])
        sobrenomes = ' '.join(partes_nome[1:])
        return primeiros_nomes, sobrenomes
    else:
        return '', ''

In [27]:
dados[0].get("Nome")

'Jo�o da Silva'

In [42]:
string= 'Antonio Carvalho da Silva'
nome, sobrenome = obter_nomes(string)

In [44]:
sobrenome

'Silva'

In [23]:
primeiro_nome, sobrenome = obter_nomes(dados[0].get("Nome"))

TypeError: expected string or bytes-like object

In [22]:
for paciente in dados:
    primeiro_nome, sobrenome = obter_nomes(paciente["Nome"])
    paciente_fhir = {
        "resourceType": "Patient",
        "identifier": [{paciente['CPF']}],
        "name": [{
            "given": primeiro_nome,
            "family": sobrenome
            }
        ],
        "telecom": [{"system": "phone", "value": paciente['Telefone']}],
        "birthDate": datetime.strptime(paciente['Data de Nascimento'],
                                        "%d/%m/%Y").strftime("%Y-%m-%d"),
        "gender": parse_gender(paciente['Genero']),
        "address": [{"country": paciente['Pais de Nascimento']}]
    }
    print(paciente_fhir[0])

TypeError: expected string or bytes-like object